In [0]:
spark.conf.set( 
    "fs.azure.account.key.goodreadsreviews60106136.dfs.core.windows.net", 
    "luY1l42vMBcNbv3kIPNPZtKJivnWX1bPA0NaFzWKMK4wXeKcx9So4xY8D9hLSiZKHlP2rOk66snF+AStEL7pWA==" 
)

In [0]:
books = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60106136.dfs.core.windows.net/processed/books/"
)
# Load the authors dataset from the silver layer
authors = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60106136.dfs.core.windows.net/processed/authors/"
)
# Display the first few records to confirm the data was loaded correctly
books.show(5)
authors.show(5)
# Display the columns and their data types to verify the schema
books.printSchema()
authors.printSchema()

+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+------

In [0]:
from pyspark.sql.functions import col, length, trim, count, when
# Read raw (uncleaned) reviews from the silver layer
reviews = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60106136.dfs.core.windows.net/processed/reviews/"
)
# Peek at rows and schema
reviews.show(5, truncate=False)
reviews.printSchema()

# Basic profiling: counts and potential issues
total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) =="") ).count()

print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id: {null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")

+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import col, trim, length
# Start from the existing Parquet-loaded DataFrame
# (Assumes you already did: reviews = spark.read.parquet(".../processed/reviews/"))
df = reviews
# 1) Drop rows missing critical keys
df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)
# 2) Enforce rating to be integer in [1..5]
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)
# 3) Normalize text; drop empty or ultra-short reviews (<10 chars after trim)
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)
# 4) De-duplicate by review_id (keep arbitrary first; refine if you have timestamps)
df = df.dropDuplicates(["review_id"])
# 5) Select final shape
reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text"
)


In [0]:
# Write the cleaned reviews back to the silver layer (overwrite)
reviews_clean.write.mode("overwrite").parquet(
"abfss://lakehouse@goodreadsreviews60106136.dfs.core.windows.net/processed/reviews/"
)
# Sanity check: re-read from disk and inspect schema and a few rows
reviews_verified = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60106136.dfs.core.windows.net/processed/reviews/"
)
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"Written cleaned rows: {reviews_verified.count()}")

root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)

+--------------------------------+--------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
books = spark.read.parquet("abfss://lakehouse@goodreadsreviews60106136.dfs.core.windows.net/processed/books/")
authors = spark.read.parquet("abfss://lakehouse@goodreadsreviews60106136.dfs.core.windows.net/processed/authors/")
book_authors = spark.read.parquet("abfss://lakehouse@goodreadsreviews60106136.dfs.core.windows.net/processed/book_authors/")
reviews_clean = spark.read.parquet("abfss://lakehouse@goodreadsreviews60106136.dfs.core.windows.net/processed/reviews/")


In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# Create synthetic bridge using existing book_id and author_id
# If you have the same number of books and authors, this works fine
book_authors = books.select("book_id") \
    .withColumn("author_id", monotonically_increasing_id())

# You can also check:
book_authors.show(5)


+-------+---------+
|book_id|author_id|
+-------+---------+
|5333265|        0|
|1333909|        1|
|7327624|        2|
|6066819|        3|
| 287140|        4|
+-------+---------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import col

# Alias tables
reviews_clean = reviews_clean.alias("r")
books = books.alias("b")
book_authors = book_authors.alias("ba")
authors = authors.alias("a")

# Perform joins using the aliases explicitly
gold_df = (
    reviews_clean.alias("r")
    .join(books.alias("b"), col("r.book_id") == col("b.book_id"), "inner")
    .join(book_authors.alias("ba"), col("b.book_id") == col("ba.book_id"), "inner")
    .join(authors.alias("a"), col("ba.author_id") == col("a.author_id"), "inner")
)


In [0]:
# Select only the needed columns
gold_df = gold_df.select(
    col("r.review_id"),
    col("r.book_id"),
    col("b.title"),
    col("a.author_id"),
    col("a.name"),
    col("r.user_id"),
    col("r.rating"),
    col("r.review_text"),
    col("b.language_code").alias("language"),
    col("b.text_reviews_count").alias("n_votes"),
    col("b.publication_year").alias("date_added")
)

In [0]:
gold_df.printSchema()
gold_df.show(5)

root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: string (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------+--------+--------------------+---------+----------------+--------------------+------+--------------------+--------+-------+----------+
|           review_id| book_id|               title|author_id|            name|             user_id|rating|         review_text|language|n_votes|date_added|
+--------------------+--------+--------------------+---------+----------------+--------------------+------+--------------------+--------+-------+----------+
|d11689f4782ff7313...| 8696320|       The Egg and I|    30753|    Jason Illian|50feca56

In [0]:
# Define path to gold zone
gold_path = "abfss://lakehouse@goodreadsreviews60106136.dfs.core.windows.net/gold/curated_reviews"

# Write as Delta table (overwrite for safety while testing)
gold_df.write.format("delta").mode("overwrite").save(gold_path)


In [0]:
# Register directly as a table
gold_df.write.format("delta").mode("overwrite").saveAsTable("curated_reviews")


In [0]:
spark.sql("SELECT * FROM curated_reviews LIMIT 5").show()

+--------------------+--------+--------------------+---------+----------------+--------------------+------+--------------------+--------+-------+----------+
|           review_id| book_id|               title|author_id|            name|             user_id|rating|         review_text|language|n_votes|date_added|
+--------------------+--------+--------------------+---------+----------------+--------------------+------+--------------------+--------+-------+----------+
|d11689f4782ff7313...| 8696320|       The Egg and I|    30753|    Jason Illian|50feca561634966e9...|     5|Sometime we forge...|     eng|     22|      1945|
|173a832185aa9fa4c...|22738563|We Should All Be ...|     8625|  Sandy Ransford|47f74a5e853959035...|     3|I thought it was ...|     eng|   5388|      2014|
|ffd106ec0ea80787f...| 6398385|Raphael (Vampires...|    29524|   Robert Eversz|1703053f9bbec94db...|     4|If you have not s...|     eng|    574|      2009|
|a1e0f4abdae5b6f7c...|  125507|The True Story of...|   169